### Cascading Classification

In [1]:
from __future__ import absolute_import, division, print_function, unicode_literals

#try:
    # %tensorflow_version only exists in Colab.
#    %tensorflow_version 2.0
#except Exception:
#    pass

# TensorFlow and tf.keras
import tensorflow as tf
from tensorflow import keras
# Helper libraries
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.utils import shuffle
from sklearn.metrics import confusion_matrix, multilabel_confusion_matrix

tf.config.experimental.set_visible_devices([], 'GPU')

AUTOTUNE = tf.data.experimental.AUTOTUNE
import IPython.display as display


import os
from os import listdir
from os.path import isfile, join
import pickle
import random
#importing module
import sys
sys.path.insert(0, '../data')
from datahandler_cascading import create_dataset

# Helper libraries
import numpy as np
import matplotlib.pyplot as plt



print(tf.__version__)

2.1.0


In [2]:
with open('../data/filenames.pkl', 'rb') as infile:
    filenames = pickle.load(infile)

with open('../data/labels.pkl', 'rb') as infile2:
    labels = pickle.load(infile2)
    
print(len(filenames), len(labels))
df = pd.concat([pd.Series(filenames, name='filenames'), pd.Series(labels, name='labels')], axis=1)

24999 24999


In [3]:
with open('../data/filenames_level2.pkl', 'rb') as infile:
    filenames2 = pickle.load(infile)

with open('../data/labels_level2.pkl', 'rb') as infile2:
    labels2 = pickle.load(infile2)

print(len(filenames2), len(labels2))
df2 = pd.concat([pd.Series(filenames2, name='filenames'), pd.Series(labels2, name='labels2')], axis=1)

24998 24998


In [4]:
df = pd.merge(df, df2, on='filenames')

In [5]:
df

,filenames,labels,labels2
0,..\..\data_tate\A00001_8.jpg,"[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
1,..\..\data_tate\A00002_8.jpg,"[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, ...","[0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, ..."
2,..\..\data_tate\A00003_8.jpg,"[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, ..."
3,..\..\data_tate\A00004_8.jpg,"[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, ..."
4,..\..\data_tate\A00005_8.jpg,"[1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 0.0, ...","[0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, ..."
...,...,...,...
24993,..\..\data_tate\T13856_8.jpg,"[0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
24994,..\..\data_tate\T13858_8.jpg,"[0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
24995,..\..\data_tate\T13863_8.jpg,"[1.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
24996,..\..\data_tate\T13864_8.jpg,"[0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."


In [6]:
class Convolution(tf.keras.layers.Layer):
    def __init__(self, filters, shape=(224,224,3), kernel_size=3,
                 pool_size=(2,2), activation='relu', padding='same'):
        super(Convolution, self).__init__()
        self.input_layer = tf.keras.layers.InputLayer(
            input_shape = shape,
        )
        self.conv_layer = tf.keras.layers.Conv2D(
            filters = filters[0], 
            padding = padding,
            kernel_size = kernel_size,
            activation = activation,
            )
        self.conv_layer1 = tf.keras.layers.Conv2D(
            filters = filters[1],
            padding = padding,
            kernel_size = kernel_size,
            activation = activation,
            )
        self.conv_layer2 = tf.keras.layers.Conv2D(
            filters = filters[1],
            padding = padding,
            kernel_size = kernel_size,
            activation = activation,
            )
        self.nn_pooling = tf.keras.layers.MaxPooling2D(
            pool_size = pool_size,
            padding = padding,
            strides = (2,2)
            )
        self.nn_dropout = tf.keras.layers.Dropout(0.2)
        self.nn_batchnorm = tf.keras.layers.BatchNormalization(axis=-1)
        self.nn_batchnorm1 = tf.keras.layers.BatchNormalization(axis=-1)
        self.nn_batchnorm2 = tf.keras.layers.BatchNormalization(axis=-1)

        
    def call(self, input_features):
        activation = self.input_layer(input_features)
        activation = self.conv_layer(activation)
        activation = self.nn_batchnorm(activation)
        activation = self.nn_pooling(activation)
        #print(activation.shape)
        activation = self.conv_layer1(activation)
        activation = self.nn_batchnorm1(activation)
        activation = self.nn_pooling(activation)
        #print(activation.shape)
        activation = self.conv_layer2(activation)
        activation = self.nn_batchnorm2(activation)
        activation = self.nn_pooling(activation)
        #print(activation.shape)
        return activation


class FC(tf.keras.layers.Layer):
    def __init__(self, filters):
        super(FC, self).__init__()
        self.flat = tf.keras.layers.GlobalAveragePooling2D()
        self.dense = tf.keras.layers.Dense(filters[0], activation='relu')
        self.dense1 = tf.keras.layers.Dense(filters[1], activation='relu')
        self.out = tf.keras.layers.Dense(filters[2], activation='sigmoid')
        self.dropout = tf.keras.layers.Dropout(0.2)
        
    def call(self, input_features):
        #print(input_features)
        activation = self.flat(input_features)
        activation = self.dense(activation)
        activation = self.dropout(activation)
        activation = self.dense1(activation)
        #print(activation)
        activation = self.dropout(activation)
        activation = self.out(activation)
        #print(activation)
        return activation
    
    
class Embed(tf.keras.layers.Layer):
    def __init__(self, emb_dim=28):
        super(Embed, self).__init__()
        self.embed = tf.keras.layers.Embedding(15, emb_dim*emb_dim, input_length=1)
        self.reshape = tf.keras.layers.Reshape((emb_dim,emb_dim))
        
        
    def call(self, input_features):
        #print(input_features.shape)
        activation = tf.argmax(input_features, axis=1)
        #print(activation.shape)
        activation = self.embed(activation)
        #print(activation.shape)
        activation = self.reshape(activation)
        return activation
    

class FullModel(tf.keras.Model):
    def __init__(self, emb_dim=28, filters_conv=[64,128], filters_dense=[2000,500,15], filters_dense1=[2000,500,141]):
        super(FullModel, self).__init__()
        self.convolution = Convolution(filters_conv)
        self.dense = FC(filters_dense)
        self.embed = Embed(emb_dim)
        self.dense1 = FC(filters_dense1)

    def call(self, input_features):
        activation = self.convolution(input_features)
        activation_ = self.dense(activation)
        #print(activation.shape)
        activation1 = self.embed(activation_)
        activation1 = tf.expand_dims(activation1, axis=3)
        #print(activation.shape, activation1.shape)
        activation2 = tf.concat([activation, activation1], 3)#do something that puts the conv and the embedding together
        #print(activation2)
        activation2 = self.dense1(activation2)
        #print(activation2)
        return [activation_, activation2]

class PretrainedModel(tf.keras.Model):
    def __init__(self, emb_dim=7, filters_dense=[2000,500,15], filters_dense1=[2000,500,141]):
        super(PretrainedModel, self).__init__()
        self.convolution = tf.keras.applications.vgg16.VGG16(include_top=False, weights='imagenet', input_shape=(224,224,3))
        self.convolution.trainable = False
        self.dense = FC(filters_dense)
        self.embed = Embed(emb_dim)
        self.dense1 = FC(filters_dense1)

    def call(self, input_features):
        activation = self.convolution(input_features)
        activation_ = self.dense(activation)
        #print(activation.shape)
        activation1 = self.embed(activation_)
        activation1 = tf.expand_dims(activation1, axis=3)
        #print(activation.shape, activation1.shape)
        activation2 = tf.concat([activation, activation1], 3)#do something that puts the conv and the embedding together
        #print(activation2)
        activation2 = self.dense1(activation2)
        #print(activation2)
        return [activation_, activation2]



model = FullModel()

pretrained_model = PretrainedModel()

model.build(input_shape=(32, 224, 224, 3))
pretrained_model.build(input_shape=(32, 224, 224, 3))
#final_model.build(input_shape=(32,224,224,4))
print(model.summary(), pretrained_model.summary())

Model: "full_model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
convolution (Convolution)    multiple                  224512    
_________________________________________________________________
fc (FC)                      multiple                  1266015   
_________________________________________________________________
embed (Embed)                multiple                  11760     
_________________________________________________________________
fc_1 (FC)                    multiple                  1331141   
Total params: 2,833,428
Trainable params: 2,832,788
Non-trainable params: 640
_________________________________________________________________
Model: "pretrained_model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
vgg16 (Model)                (None, 7, 7, 512)         14714688  
__________________

In [7]:
def loss(model, x_train, y_train, _y_train):
    loss_object = tf.keras.losses.BinaryCrossentropy(from_logits=True)
    loss_binary = tf.keras.losses.BinaryCrossentropy(from_logits=True)
    intermidiate_loss = loss_binary(model(x_train)[0], y_train)
    reconstruction_error = loss_object(model(x_train)[1], _y_train)
    return tf.math.add(reconstruction_error,intermidiate_loss)


def train(loss, model, opt, x_train=None, y_train=None, _y_train=None):
    with tf.GradientTape() as tape:
        gradients = tape.gradient(loss(model, x_train, y_train, _y_train), model.trainable_variables)
        gradient_variables = zip(gradients, model.trainable_variables)
        opt.apply_gradients(gradient_variables)

In [8]:
train_x = list(df['filenames'][:18000])
train_y = list(df['labels'][:18000])
train_y_ = list(df['labels2'][:18000])
val_x = list(df['filenames'][18000:21000])
val_y = list(df['labels'][18000:21000])
val_y_ = list(df['labels2'][18000:21000])
test_x = list(df['filenames'][21000:])
test_y = list(df['labels'][21000:])
test_y_ = list(df['labels2'][21000:])

with open('../data/train_test_split_hierarchical.pkl', 'wb') as infile:
    pickle.dump((train_x, train_y, train_y_, val_x, val_y, val_y_, test_x, test_y, test_y_), infile)

In [9]:
with open('../data/train_test_split_hierarchical.pkl', 'rb') as infile:
    train_x, train_y, train_y_, val_x, val_y, val_y_, test_x, test_y, test_y_ = pickle.load(infile)

In [10]:
train_generator = create_dataset(train_x, train_y, train_y_)
val_generator = create_dataset(val_x, val_y, val_y_)
test_generator = create_dataset(test_x, test_y, test_y_, BATCH_SIZE=1)

In [11]:
def macro_f1(y, y_hat, thresh=0.5):
    """Compute the macro F1-score on a batch of observations (average F1 across labels)
    
    Args:
        y (int32 Tensor): labels array of shape (BATCH_SIZE, N_LABELS)
        y_hat (float32 Tensor): probability matrix from forward propagation of shape (BATCH_SIZE, N_LABELS)
        thresh: probability value above which we predict positive
        
    Returns:
        macro_f1 (scalar Tensor): value of macro F1 for the batch
    """
    y_pred = tf.cast(tf.greater(y_hat, thresh), tf.float32)
    tp = tf.cast(tf.math.count_nonzero(y_pred * y, axis=0), tf.float32)
    fp = tf.cast(tf.math.count_nonzero(y_pred * (1 - y), axis=0), tf.float32)
    fn = tf.cast(tf.math.count_nonzero((1 - y_pred) * y, axis=0), tf.float32)
    f1 = 2*tp / (2*tp + fn + fp + 1e-16)
    macro_f1 = tf.reduce_mean(f1)
    return macro_f1

def accuracy(y, y_hat):
    y_pred = tf.cast(tf.greater(y_hat, 0.5), tf.float32)
    tp = tf.cast(tf.math.count_nonzero(y_pred * y, axis=0), tf.float32)
    fp = tf.cast(tf.math.count_nonzero(y_pred * (1 - y), axis=0), tf.float32)
    fn = tf.cast(tf.math.count_nonzero((1 - y_pred) * y, axis=0), tf.float32)
    tn = tf.cast(tf.math.count_nonzero((1 - y_pred) * (1 - y), axis=0), tf.float32)
    accuracy = (tp + tn) / (tp + tn + fn + fp)
    accuracy = tf.reduce_mean(accuracy)
    return accuracy

def precision(y, y_hat):
    y_pred = tf.cast(tf.greater(y_hat, 0.5), tf.float32)
    tp = tf.cast(tf.math.count_nonzero(y_pred * y, axis=0), tf.float32)
    fp = tf.cast(tf.math.count_nonzero(y_pred * (1 - y), axis=0), tf.float32)
    pres = tp / (tp + fp + 1e-16)
    precision = tf.reduce_mean(pres)
    return precision

def recall(y, y_hat):
    y_pred = tf.cast(tf.greater(y_hat, 0.5), tf.float32)
    tp = tf.cast(tf.math.count_nonzero(y_pred * y, axis=0), tf.float32)
    fn = tf.cast(tf.math.count_nonzero((1 - y_pred) * y, axis=0), tf.float32)
    rec = tp / (tp + fn + 1e-16)
    recall = tf.reduce_mean(rec)
    return recall

In [12]:
tf.compat.v1.enable_eager_execution()
# Keep results for plotting
train_loss_results = []
train_accuracy_results = []
opt = tf.keras.optimizers.Adam() #learning_rate=learning_rate: will add in future version
num_epochs = 2

model = PretrainedModel()

for i in range(num_epochs):
    for batch in range(len(train_x)//56):
        features, labels1, labels2 = next(iter(train_generator))
        epoch_loss_avg = tf.keras.metrics.Mean()
        epoch_accuracy = tf.keras.metrics.Accuracy()
        train(loss, model, opt, x_train=features, y_train=labels1, _y_train=labels2)
        loss_values = loss(model, x_train=features, y_train=labels1, _y_train=labels2)
        print(loss_values)
        print('precision', precision(model(features)[1], labels2))
        print('recall', recall(model(features)[1], labels2))
        print('accuracy', accuracy(model(features)[1], labels2))
        print('f1', macro_f1(model(features)[1], labels2))
        train_loss_results.append(loss_values)
        print('iter end', batch)
    print('epoch end', i)
    
# Save the weights
model.save_weights('./cascading_model_VGG.h5')

tf.Tensor(1.349815845489502, shape=(), dtype=float64)
precision tf.Tensor(0.26950353, shape=(), dtype=float32)
recall tf.Tensor(0.042553186, shape=(), dtype=float32)
accuracy tf.Tensor(0.5, shape=(), dtype=float32)
f1 tf.Tensor(0.062593766, shape=(), dtype=float32)
iter end 0
tf.Tensor(1.3258989453315735, shape=(), dtype=float64)
precision tf.Tensor(0.26950353, shape=(), dtype=float32)
recall tf.Tensor(0.036347516, shape=(), dtype=float32)
accuracy tf.Tensor(0.5, shape=(), dtype=float32)
f1 tf.Tensor(0.05335098, shape=(), dtype=float32)
iter end 1
tf.Tensor(1.3139403462409973, shape=(), dtype=float64)
precision tf.Tensor(0.26595744, shape=(), dtype=float32)
recall tf.Tensor(0.036980752, shape=(), dtype=float32)
accuracy tf.Tensor(0.5, shape=(), dtype=float32)
f1 tf.Tensor(0.052186515, shape=(), dtype=float32)
iter end 2
tf.Tensor(1.2944188714027405, shape=(), dtype=float64)
precision tf.Tensor(0.28014183, shape=(), dtype=float32)
recall tf.Tensor(0.042806488, shape=(), dtype=float32)
a

precision tf.Tensor(0.43676123, shape=(), dtype=float32)
recall tf.Tensor(0.035207704, shape=(), dtype=float32)
accuracy tf.Tensor(0.35046595, shape=(), dtype=float32)
f1 tf.Tensor(0.057925664, shape=(), dtype=float32)
iter end 29
tf.Tensor(1.2931703925132751, shape=(), dtype=float64)
precision tf.Tensor(0.42285264, shape=(), dtype=float32)
recall tf.Tensor(0.037993923, shape=(), dtype=float32)
accuracy tf.Tensor(0.34660274, shape=(), dtype=float32)
f1 tf.Tensor(0.061464384, shape=(), dtype=float32)
iter end 30
tf.Tensor(1.2934303879737854, shape=(), dtype=float64)
precision tf.Tensor(0.39004362, shape=(), dtype=float32)
recall tf.Tensor(0.03850051, shape=(), dtype=float32)
accuracy tf.Tensor(0.34529406, shape=(), dtype=float32)
f1 tf.Tensor(0.06333045, shape=(), dtype=float32)
iter end 31
tf.Tensor(1.3041976690292358, shape=(), dtype=float64)
precision tf.Tensor(0.42594564, shape=(), dtype=float32)
recall tf.Tensor(0.036347516, shape=(), dtype=float32)
accuracy tf.Tensor(0.32877317, s

precision tf.Tensor(0.54137117, shape=(), dtype=float32)
recall tf.Tensor(0.04052685, shape=(), dtype=float32)
accuracy tf.Tensor(0.27277806, shape=(), dtype=float32)
f1 tf.Tensor(0.06722843, shape=(), dtype=float32)
iter end 58
tf.Tensor(1.2971540689468384, shape=(), dtype=float64)
precision tf.Tensor(0.46394798, shape=(), dtype=float32)
recall tf.Tensor(0.035840932, shape=(), dtype=float32)
accuracy tf.Tensor(0.26986796, shape=(), dtype=float32)
f1 tf.Tensor(0.0590169, shape=(), dtype=float32)
iter end 59
tf.Tensor(1.295643389225006, shape=(), dtype=float64)
precision tf.Tensor(0.51962173, shape=(), dtype=float32)
recall tf.Tensor(0.038627155, shape=(), dtype=float32)
accuracy tf.Tensor(0.2695796, shape=(), dtype=float32)
f1 tf.Tensor(0.06373263, shape=(), dtype=float32)
iter end 60
tf.Tensor(1.288474977016449, shape=(), dtype=float64)
precision tf.Tensor(0.4727457, shape=(), dtype=float32)
recall tf.Tensor(0.039640326, shape=(), dtype=float32)
accuracy tf.Tensor(0.28589153, shape=()

precision tf.Tensor(0.49940902, shape=(), dtype=float32)
recall tf.Tensor(0.038120568, shape=(), dtype=float32)
accuracy tf.Tensor(0.26840633, shape=(), dtype=float32)
f1 tf.Tensor(0.06392379, shape=(), dtype=float32)
iter end 87
tf.Tensor(1.2878493666648865, shape=(), dtype=float64)
precision tf.Tensor(0.5101655, shape=(), dtype=float32)
recall tf.Tensor(0.04128673, shape=(), dtype=float32)
accuracy tf.Tensor(0.27694997, shape=(), dtype=float32)
f1 tf.Tensor(0.06869391, shape=(), dtype=float32)
iter end 88
tf.Tensor(1.2975966930389404, shape=(), dtype=float64)
precision tf.Tensor(0.46300235, shape=(), dtype=float32)
recall tf.Tensor(0.038247216, shape=(), dtype=float32)
accuracy tf.Tensor(0.2789399, shape=(), dtype=float32)
f1 tf.Tensor(0.06264903, shape=(), dtype=float32)
iter end 89
tf.Tensor(1.2807962894439697, shape=(), dtype=float64)
precision tf.Tensor(0.5326241, shape=(), dtype=float32)
recall tf.Tensor(0.043186422, shape=(), dtype=float32)
accuracy tf.Tensor(0.28076643, shape=

tf.Tensor(1.2915539145469666, shape=(), dtype=float64)
precision tf.Tensor(0.47748765, shape=(), dtype=float32)
recall tf.Tensor(0.037487336, shape=(), dtype=float32)
accuracy tf.Tensor(0.2764448, shape=(), dtype=float32)
f1 tf.Tensor(0.061941687, shape=(), dtype=float32)
iter end 116
tf.Tensor(1.2860115766525269, shape=(), dtype=float64)
precision tf.Tensor(0.5186761, shape=(), dtype=float32)
recall tf.Tensor(0.04255319, shape=(), dtype=float32)
accuracy tf.Tensor(0.27589998, shape=(), dtype=float32)
f1 tf.Tensor(0.07037432, shape=(), dtype=float32)
iter end 117
tf.Tensor(1.2959416508674622, shape=(), dtype=float64)
precision tf.Tensor(0.47281322, shape=(), dtype=float32)
recall tf.Tensor(0.035460994, shape=(), dtype=float32)
accuracy tf.Tensor(0.26864007, shape=(), dtype=float32)
f1 tf.Tensor(0.058562454, shape=(), dtype=float32)
iter end 118
tf.Tensor(1.2951045036315918, shape=(), dtype=float64)
precision tf.Tensor(0.47948822, shape=(), dtype=float32)
recall tf.Tensor(0.03647417, sh

...

f1 tf.Tensor(0.06516309, shape=(), dtype=float32)
iter end 198
tf.Tensor(1.2831249237060547, shape=(), dtype=float64)
precision tf.Tensor(0.5531915, shape=(), dtype=float32)
recall tf.Tensor(0.044199593, shape=(), dtype=float32)
accuracy tf.Tensor(0.27628613, shape=(), dtype=float32)
f1 tf.Tensor(0.07321504, shape=(), dtype=float32)
iter end 199
tf.Tensor(1.3024174571037292, shape=(), dtype=float64)
precision tf.Tensor(0.44984803, shape=(), dtype=float32)
recall tf.Tensor(0.03508105, shape=(), dtype=float32)
accuracy tf.Tensor(0.26376602, shape=(), dtype=float32)
f1 tf.Tensor(0.05613489, shape=(), dtype=float32)
iter end 200
tf.Tensor(1.2885228991508484, shape=(), dtype=float64)
precision tf.Tensor(0.4840425, shape=(), dtype=float32)
recall tf.Tensor(0.039513677, shape=(), dtype=float32)
accuracy tf.Tensor(0.27106878, shape=(), dtype=float32)
f1 tf.Tensor(0.06468646, shape=(), dtype=float32)
iter end 201
tf.Tensor(1.2909284234046936, shape=(), dtype=float64)
precision tf.Tensor(0.47044

accuracy tf.Tensor(0.27699503, shape=(), dtype=float32)
f1 tf.Tensor(0.07204874, shape=(), dtype=float32)
iter end 227
tf.Tensor(1.2939499020576477, shape=(), dtype=float64)
precision tf.Tensor(0.46173167, shape=(), dtype=float32)
recall tf.Tensor(0.039513677, shape=(), dtype=float32)
accuracy tf.Tensor(0.28003225, shape=(), dtype=float32)
f1 tf.Tensor(0.064209364, shape=(), dtype=float32)
iter end 228
tf.Tensor(1.2906494140625, shape=(), dtype=float64)
precision tf.Tensor(0.4219858, shape=(), dtype=float32)
recall tf.Tensor(0.03748733, shape=(), dtype=float32)
accuracy tf.Tensor(0.27228758, shape=(), dtype=float32)
f1 tf.Tensor(0.061454277, shape=(), dtype=float32)
iter end 229
tf.Tensor(1.3044669032096863, shape=(), dtype=float64)
precision tf.Tensor(0.43026006, shape=(), dtype=float32)
recall tf.Tensor(0.03444782, shape=(), dtype=float32)
accuracy tf.Tensor(0.27080253, shape=(), dtype=float32)
f1 tf.Tensor(0.054861385, shape=(), dtype=float32)
iter end 230
tf.Tensor(1.29068797826766

recall tf.Tensor(0.036727455, shape=(), dtype=float32)
accuracy tf.Tensor(0.2760086, shape=(), dtype=float32)
f1 tf.Tensor(0.060605746, shape=(), dtype=float32)
iter end 256
tf.Tensor(1.2904762625694275, shape=(), dtype=float64)
precision tf.Tensor(0.5248227, shape=(), dtype=float32)
recall tf.Tensor(0.03913374, shape=(), dtype=float32)
accuracy tf.Tensor(0.27681643, shape=(), dtype=float32)
f1 tf.Tensor(0.06576622, shape=(), dtype=float32)
iter end 257
tf.Tensor(1.306112289428711, shape=(), dtype=float64)
precision tf.Tensor(0.4397163, shape=(), dtype=float32)
recall tf.Tensor(0.03368794, shape=(), dtype=float32)
accuracy tf.Tensor(0.26802608, shape=(), dtype=float32)
f1 tf.Tensor(0.05490012, shape=(), dtype=float32)
iter end 258
tf.Tensor(1.3020133972167969, shape=(), dtype=float64)
precision tf.Tensor(0.44929078, shape=(), dtype=float32)
recall tf.Tensor(0.03495441, shape=(), dtype=float32)
accuracy tf.Tensor(0.2738686, shape=(), dtype=float32)
f1 tf.Tensor(0.055343945, shape=(), dt

precision tf.Tensor(0.46335694, shape=(), dtype=float32)
recall tf.Tensor(0.0371074, shape=(), dtype=float32)
accuracy tf.Tensor(0.27598634, shape=(), dtype=float32)
f1 tf.Tensor(0.061037455, shape=(), dtype=float32)
iter end 285
tf.Tensor(1.2871372699737549, shape=(), dtype=float64)
precision tf.Tensor(0.49160755, shape=(), dtype=float32)
recall tf.Tensor(0.040780146, shape=(), dtype=float32)
accuracy tf.Tensor(0.27578795, shape=(), dtype=float32)
f1 tf.Tensor(0.067479834, shape=(), dtype=float32)
iter end 286
tf.Tensor(1.293959379196167, shape=(), dtype=float64)
precision tf.Tensor(0.4775414, shape=(), dtype=float32)
recall tf.Tensor(0.037107397, shape=(), dtype=float32)
accuracy tf.Tensor(0.26981378, shape=(), dtype=float32)
f1 tf.Tensor(0.060737684, shape=(), dtype=float32)
iter end 287
tf.Tensor(1.2909765243530273, shape=(), dtype=float64)
precision tf.Tensor(0.4716312, shape=(), dtype=float32)
recall tf.Tensor(0.039007094, shape=(), dtype=float32)
accuracy tf.Tensor(0.2769381, sh

tf.Tensor(1.294844627380371, shape=(), dtype=float64)
precision tf.Tensor(0.42021278, shape=(), dtype=float32)
recall tf.Tensor(0.03596758, shape=(), dtype=float32)
accuracy tf.Tensor(0.27408117, shape=(), dtype=float32)
f1 tf.Tensor(0.05743515, shape=(), dtype=float32)
iter end 314
tf.Tensor(1.2905532717704773, shape=(), dtype=float64)
precision tf.Tensor(0.4574468, shape=(), dtype=float32)
recall tf.Tensor(0.03774063, shape=(), dtype=float32)
accuracy tf.Tensor(0.2766233, shape=(), dtype=float32)
f1 tf.Tensor(0.06266927, shape=(), dtype=float32)
iter end 315
tf.Tensor(1.2942193150520325, shape=(), dtype=float64)
precision tf.Tensor(0.47074467, shape=(), dtype=float32)
recall tf.Tensor(0.037613984, shape=(), dtype=float32)
accuracy tf.Tensor(0.28102106, shape=(), dtype=float32)
f1 tf.Tensor(0.061925, shape=(), dtype=float32)
iter end 316
tf.Tensor(1.3076615929603577, shape=(), dtype=float64)
precision tf.Tensor(0.40780142, shape=(), dtype=float32)
recall tf.Tensor(0.033181354, shape=(

In [13]:
# Create a new model instance
model = PretrainedModel()

# Restore the weights
model.load_weights('./cascading_model_VGG.h5')

# Evaluate the model
#loss,acc = model.evaluate(test_generator, verbose=2)
#print("Restored model, accuracy: {:5.2f}%".format(100*acc))

ValueError: You are trying to load a weight file containing 4 layers into a model with 1 layers.

In [ ]:
import pickle
predictions = []
for i in range(1):
    for batch in range(len(test_x)):
        features, labels1, labels2 = next(iter(test_generator))
        epoch_loss_avg = tf.keras.metrics.Mean()
        epoch_accuracy = tf.keras.metrics.Accuracy()
        loss_values = loss(model, x_train=features, y_train=labels1, _y_train=labels2)
        predictions.append((model(features)[1].numpy(), labels2.numpy()))
        print('iter end', batch)
    print('epoch end', i)
    
with open('./results/predictions_cascading.pkl', 'wb') as outfile:
    pickle.dump(predictions, outfile)

In [ ]:
predictions